In [1]:
import os
import re
from nltk.tokenize import sent_tokenize
from scipy.signal import decimate
from tdmh import *
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
ana = SentimentIntensityAnalyzer()
import sklearn
from sklearn.preprocessing import minmax_scale
from scipy.fftpack import dct, idct
import pandas as pd
from math import ceil
import stanza
import numpy as np
from tqdm import tqdm


#import stanza
#stanza.download('en') # download English model
#nlp = stanza.Pipeline('en') # initialize English neural pipeline

import seaborn as sns
import matplotlib.pyplot as plt

nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

2023-01-09 16:50:56 WARNING: Can not find mwt: default from official model list. Ignoring it.
2023-01-09 16:50:56 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2023-01-09 16:50:56 INFO: Use device: cpu
2023-01-09 16:50:56 INFO: Loading: tokenize
2023-01-09 16:50:56 INFO: Loading: pos
2023-01-09 16:50:56 INFO: Loading: lemma
2023-01-09 16:50:56 INFO: Done loading processors!


In [2]:
def get_text_as_string(path):
    with open(path , encoding = 'utf-8') as text_file:
        full_text = text_file.read()
        full_text = re.sub( '\s+' , ' ' , full_text)
        return full_text
file = 'AStrangeCrime.txt' 
full_text = get_text_as_string( file )
#print(full_text)

In [3]:
def get_sentences(full_text):
    sentences = sent_tokenize(full_text.strip())
    return sentences

sentences = get_sentences(full_text)
print( len(sentences) )
print( sentences[0:3] )

3309
['BOOK I A STRANGE CRIME CHAPTER I.', 'RED LIGHT.', 'Mr. Gryce was melancholy.']


In [4]:
# doc = nlp(full_text)
# for sent in doc.sentences:
#     new_sentence = ''
#     for word in sent.words:
#         new_sentence += f'{word.lemma} '  

In [ ]:
lemmatised_sentences = []


with tqdm(total=len(sentences)) as t:
    for s in sentences:

        t.update(1)

        doc = nlp(s)
        for sent in doc.sentences:
            new_sentence = ''
            for word in sent.words:
                new_sentence += f'{word.lemma} '
            lemmatised_sentences.append(new_sentence)

        

  8%|███▏                                    | 261/3309 [00:16<03:20, 15.22it/s]

In [ ]:
import pandas as pd
words = pd.read_csv('labMT.tsv' , sep = '\t')
print(words.columns)
wordscores = dict()
for i,row in words.iterrows():
    wordscores[row['word']] = row['happiness_average']


In [ ]:
import pyhmeter

In [ ]:
def get_sentiment( sentences ):

    all_scores = []
    if len(sentences)>0:
        for s in sentences:
            words = re.split( r'\s+' , s)
            h = pyhmeter.HMeter( words , wordscores , deltah = 1.0 )
            if h.happiness_score() is not None:
                all_scores.append( h.happiness_score() )
            
    return all_scores

scores = get_sentiment(lemmatised_sentences)


## Plotting

In [ ]:
novel_time = range( 0, len(scores))
plt.figure( figsize=(12,5) )
sns.lineplot( x = novel_time , y = scores )
plt.show()

In [ ]:
print(len(novel_time))

In [ ]:
#df_temperature['SMA_10'] = df_temperature.average_temperature.rolling(10, min_periods=1).mean()


In [ ]:
def rolling_average(scores):
    all_averages = []
    window_size = math.ceil( 0.10 * len(scores) )
    start_index = 0
    end_index = window_size
    
    #print( f'{start_index}:{end_index}' )    
    subset = scores[start_index:end_index]
    avg = sum(subset)/len(subset)
    all_averages.append(avg)
    #print(len(subset))
    
    while end_index < len(scores):
        
        start_index += 1
        end_index = (start_index + window_size)
        if end_index > len(scores):
            end_index = len(scores)
            
        #print( f'{start_index}:{end_index}' )
            
        subset = scores[start_index:end_index]
        
        #print(len(subset))
        avg = sum(subset)/len(subset)
        all_averages.append(avg)
    return all_averages
        
      
ra = rolling_average(scores)
print(len(ra))


In [ ]:
scores_plot = ra
novel_time = range( 1, len(scores_plot)+1)
plt.figure( figsize=(10,5) )
ax = sns.lineplot( x = novel_time , y = scores_plot )
#ax.set_xticks(novel_time)
plt.show()

In [ ]:

x = ceil(len(ra)/100)
ra_dec = decimate(ra,x)
#print(ra_dec)
new_list = []

for i in range(0,len(ra)+2):
    if i % x == 0:
        new_list.append(ra[i])
new_list.append(ra[-1])
        
print(len(new_list))


In [ ]:
scores_plot = ra_dec
novel_time = range( 1, len(scores_plot)+1)
plt.figure( figsize=(10,5) )
ax = sns.lineplot( x = novel_time , y = scores_plot )
#ax.set_xticks(novel_time)
img_filename = re.sub( r'txt$' , 'png' , file )
plt.savefig(img_filename)